## Import and load

In [1]:
import duckdb

# Conexão em memória
con = duckdb.connect()

# Defina o caminho para o arquivo
parquet_path = "base/yellow_tripdata_2023-01.parquet"

## Number of rides

In [2]:
con.execute(f"""
    SELECT COUNT(*) AS total_rides
    FROM '{parquet_path}'
""").df()


,total_rides
0,3066766


## Distribuition of paassenger per rider

In [3]:
con.execute(f"""
    SELECT passenger_count, COUNT(*) AS num_rides
    FROM '{parquet_path}'
    GROUP BY passenger_count
    ORDER BY passenger_count
""").df()


,passenger_count,num_rides
0,0.0,51164
1,1.0,2261400
2,2.0,451536
3,3.0,106353
4,4.0,53745
5,5.0,42681
6,6.0,28124
7,7.0,6
8,8.0,13
9,9.0,1


## ride distance statistics

In [4]:
con.execute(f"""
    SELECT
        MIN(trip_distance) AS min_distance,
        MAX(trip_distance) AS max_distance,
        AVG(trip_distance) AS avg_distance,
        STDDEV(trip_distance) AS std_distance
    FROM '{parquet_path}'
""").df()


,min_distance,max_distance,avg_distance,std_distance
0,0.0,258928.15,3.847342,249.583756


## Total rece value statistcs

In [7]:
con.execute(f"""
    SELECT
        MIN(total_amount) AS min_total,
        MAX(total_amount) AS max_total,
        AVG(total_amount) AS avg_total,
        MEDIAN(total_amount) AS median_total
    FROM '{parquet_path}'
""").df()


,min_total,max_total,avg_total,median_total
0,-751.0,1169.4,27.020383,20.16


## Races by day of the month

In [10]:
con.execute(f"""
    SELECT 
        DATE(tpep_pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides
    FROM '{parquet_path}'
    GROUP BY ride_date
    ORDER BY ride_date
""").df()


,ride_date,total_rides
0,2008-12-31,2
1,2022-10-24,4
2,2022-10-25,7
3,2022-12-31,25
4,2023-01-01,76752
5,2023-01-02,65777
6,2023-01-03,85783
7,2023-01-04,95092
8,2023-01-05,101063
9,2023-01-06,102550


## Races for hour of day

In [12]:
con.execute(f"""
    SELECT 
        EXTRACT(HOUR FROM tpep_pickup_datetime) AS pickup_hour,
        COUNT(*) AS total_rides
    FROM '{parquet_path}'
    GROUP BY pickup_hour
    ORDER BY pickup_hour
""").df()


,pickup_hour,total_rides
0,0,84969
1,1,59799
2,2,42040
3,3,27438
4,4,17835
5,5,18011
6,6,43860
7,7,86877
8,8,116865
9,9,131111


## Frequency by payment type

In [13]:
con.execute(f"""
    SELECT 
        payment_type,
        COUNT(*) AS total_rides,
        AVG(total_amount) AS avg_amount
    FROM '{parquet_path}'
    GROUP BY payment_type
    ORDER BY total_rides DESC
""").df()


,payment_type,total_rides,avg_amount
0,1,2411462,28.298442
1,2,532241,23.034783
2,0,71743,29.133590
3,4,33297,2.553484
4,3,18023,10.506835


 ## Identifying outliers in distances

In [14]:
con.execute(f"""
    SELECT *
    FROM '{parquet_path}'
    WHERE trip_distance > 50
    ORDER BY trip_distance DESC
    LIMIT 10
""").df()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-23 17:14:00,2023-01-23 17:20:00,NaN,258928.15,NaN,None,238,239,0,13.65,0.0,0.5,3.53,0.0,1.0,21.18,NaN,NaN
1,2,2023-01-07 18:18:00,2023-01-07 18:47:00,NaN,225987.37,NaN,None,261,263,0,56.49,0.0,0.5,12.10,0.0,1.0,72.59,NaN,NaN
2,2,2023-01-15 08:33:00,2023-01-15 08:43:00,NaN,187872.33,NaN,None,229,142,0,14.02,0.0,0.5,3.60,0.0,1.0,21.62,NaN,NaN
3,2,2023-01-13 05:53:00,2023-01-13 06:02:00,NaN,116439.71,NaN,None,239,230,0,13.22,0.0,0.5,3.00,0.0,1.0,20.22,NaN,NaN
4,2,2023-01-31 07:18:00,2023-01-31 07:57:00,NaN,85543.66,NaN,None,41,87,0,39.90,0.0,0.5,7.90,0.0,1.0,51.80,NaN,NaN
5,2,2023-01-12 08:45:00,2023-01-12 09:09:00,NaN,76886.52,NaN,None,148,68,0,21.49,0.0,0.5,5.10,0.0,1.0,30.59,NaN,NaN
6,2,2023-01-01 19:40:43,2023-01-01 20:21:48,1.0,62359.52,4.0,N,79,48,2,0.00,0.0,0.0,0.00,0.0,0.0,2.50,2.5,0.0
7,2,2023-01-31 21:24:00,2023-01-31 21:28:00,NaN,52042.30,NaN,None,236,75,0,14.40,0.0,0.5,1.84,0.0,1.0,20.24,NaN,NaN
8,2,2023-01-22 11:25:00,2023-01-22 11:33:00,NaN,33205.32,NaN,None,230,43,0,12.87,0.0,0.5,3.37,0.0,1.0,20.24,NaN,NaN
9,2,2023-01-23 07:29:00,2023-01-23 07:51:00,NaN,16562.61,NaN,None,262,90,0,28.87,0.0,0.5,4.93,0.0,1.0,37.80,NaN,NaN
